In [1]:
import re
import glob
import xlwings as xw
import math
from pathlib import Path
import random
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
from matplotlib import cm
import matplotlib.pyplot as plt
import matplotlib.colors as mclr
import plotly.express as px
import statsmodels.api as sm
import pylab as py
import os, cv2, glob, tempfile
import joblib

from scipy import stats
from scipy.stats import pearsonr

import sklearn
from sklearn import datasets
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.datasets import make_blobs

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import VotingClassifier
from sklearn import metrics
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import matthews_corrcoef
make_scorer = sklearn.metrics.make_scorer
f1 = make_scorer(f1_score, pos_label=1, average="binary")
from sklearn.metrics import classification_report
from sklearn.inspection import permutation_importance
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, KFold
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.model_selection import RepeatedStratifiedKFold

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras.optimizers import Adam
from keras import utils as np_utils
from keras import models
import keras_tuner as kt
from scipy import signal

import config as config
print(config.__path__)
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras import backend as K

import argparse
# from sklearn.utils import class_weight
print(tf.__version__)
print(keras.__version__)

import shutil
import xlwings as xw

%matplotlib inline
mpl.use("Agg")

['C:\\Users\\user\\anaconda3\\lib\\site-packages\\config']
2.12.0-dev20221107
2.12.0


In [2]:
import warnings
warnings.filterwarnings("ignore")

# 1. Data Preparation

In [14]:
## input training set ## 90960 x 16 df
df_ROI_train = pd.read_csv(r"H:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\XGB_ALL\\afterModelSelection\\dbMSIn_10ppm\\df_train_dbMSIn10ppm_norm_0n1_postPredict.csv").drop(columns=["Column1"]).reset_index().drop(columns=["index"])

In [15]:
## input ext val set ## 6075 x 16 df
df_ROI_ext = pd.read_csv(r"H:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\XGB_ALL\\afterModelSelection\\dbMSIn_10ppm\\df_ext_dbMSIn10ppm_norm_0n1_postPredict.csv").drop(columns=["Column1"]).reset_index().drop(columns=["index"])

In [16]:
## input FNA set ## 88701 x 16 df
df_ROI_FNA = pd.read_csv(r"H:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\XGB_ALL\\afterModelSelection\\dbMSIn_10ppm\\df_FNA_dbMSIn10ppm_norm_0n1_postPredict.csv").drop(columns=["Column1"]).reset_index().drop(columns=["index"])

In [17]:
## input DirectInfusion set ## 6057 x 16 df
df_ROI_Di = pd.read_csv(r"H:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\XGB_ALL\\afterModelSelection\\dbMSIn_10ppm\\df_Di_dbMSIn10ppm_norm_0n1_postPredict.csv").drop(columns=["Column1"]).reset_index().drop(columns=["index"])

In [18]:
df_ROI_Di

,pixel_id,311.1684,269.2486,215.0328,295.2278,883.533,309.1704,738.5066,435.2965,250.1449,514.2846,type,predicted0n1,T,F,CM
0,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,0.005233,0.142288,0.039170,0.003879,0.077996,0.211155,0.061852,0.125873,0.467335,0.036498,0,0,0,1,TN
1,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,0.089591,0.156412,0.174829,0.001265,0.167022,0.317443,0.127993,0.145532,0.583291,0.099618,0,1,1,0,FP
2,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,0.032985,0.134206,0.034411,-0.069655,0.121979,0.212954,0.028980,0.076831,0.465178,0.056825,0,0,0,1,TN
3,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,0.081001,0.235152,0.061915,0.057514,0.129092,0.303459,0.108110,0.193013,0.550683,0.090339,0,0,0,1,TN
4,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,0.132548,0.236938,0.152597,0.080273,0.239269,0.394214,0.236241,0.204562,0.645485,0.147333,0,0,0,1,TN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6052,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S1...,-0.002046,-0.209455,0.152213,-0.039157,-0.042523,-0.056631,0.023261,0.147171,-0.089381,0.163267,1,1,1,0,TP
6053,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S1...,-0.313188,-0.217983,-0.308808,-0.299289,-0.273013,-0.303023,-0.280535,-0.277840,-0.025246,-0.251944,1,1,1,0,TP
6054,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S1...,-0.158181,-0.037213,-0.000751,-0.048526,-0.045113,-0.153125,-0.084896,-0.039408,-0.089381,0.091528,1,1,1,0,TP
6055,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S1...,-0.130111,-0.100314,0.065751,-0.107947,-0.006814,-0.127231,-0.068984,-0.070642,-0.010182,-0.018040,1,1,1,0,TP


In [19]:
df_ROI_train["Sample"] = 0
df_ROI_train["Row"] = 0
df_ROI_train["Scan"] = 0
df_ROI_train_Sample = []
df_ROI_train_Row = []
df_ROI_train_Scan = []
for i in range(len(df_ROI_train["pixel_id"])):
    df_ROI_train_Sample.append("_".join(str(df_ROI_train["pixel_id"][i]).split("_")[:-2]))
    df_ROI_train_Row.append(int(str(df_ROI_train["pixel_id"][i]).split("_")[-1]))
    df_ROI_train_Scan.append(int(str(df_ROI_train["pixel_id"][i]).split("_")[-2]))
df_ROI_train["Sample"] = df_ROI_train_Sample
df_ROI_train["Row"] = df_ROI_train_Row
df_ROI_train["Scan"] = df_ROI_train_Scan
pop_column = df_ROI_train.pop('Scan')
df_ROI_train.insert(1, 'Scan', pop_column)
pop_column = df_ROI_train.pop('Row')
df_ROI_train.insert(1, 'Row', pop_column)
pop_column = df_ROI_train.pop('Sample')
df_ROI_train.insert(1, 'Sample', pop_column)

df_ROI_ext["Sample"] = 0
df_ROI_ext["Row"] = 0
df_ROI_ext["Scan"] = 0
df_ROI_ext_Sample = []
df_ROI_ext_Row = []
df_ROI_ext_Scan = []
for i in range(len(df_ROI_ext["pixel_id"])):
    df_ROI_ext_Sample.append("_".join(str(df_ROI_ext["pixel_id"][i]).split("_")[:-2]))
    df_ROI_ext_Row.append(int(str(df_ROI_ext["pixel_id"][i]).split("_")[-1]))
    df_ROI_ext_Scan.append(int(str(df_ROI_ext["pixel_id"][i]).split("_")[-2]))
df_ROI_ext["Sample"] = df_ROI_ext_Sample
df_ROI_ext["Row"] = df_ROI_ext_Row
df_ROI_ext["Scan"] = df_ROI_ext_Scan
pop_column = df_ROI_ext.pop('Scan')
df_ROI_ext.insert(1, 'Scan', pop_column)
pop_column = df_ROI_ext.pop('Row')
df_ROI_ext.insert(1, 'Row', pop_column)
pop_column = df_ROI_ext.pop('Sample')
df_ROI_ext.insert(1, 'Sample', pop_column)

df_ROI_FNA["Sample"] = 0
df_ROI_FNA["Row"] = 0
df_ROI_FNA["Scan"] = 0
df_ROI_FNA_Sample = []
df_ROI_FNA_Row = []
df_ROI_FNA_Scan = []
for i in range(len(df_ROI_FNA["pixel_id"])):
    df_ROI_FNA_Sample.append("_".join(str(df_ROI_FNA["pixel_id"][i]).split("_")[:-2]))
    df_ROI_FNA_Row.append(int(str(df_ROI_FNA["pixel_id"][i]).split("_")[-1]))
    df_ROI_FNA_Scan.append(int(str(df_ROI_FNA["pixel_id"][i]).split("_")[-2]))
df_ROI_FNA["Sample"] = df_ROI_FNA_Sample
df_ROI_FNA["Row"] = df_ROI_FNA_Row
df_ROI_FNA["Scan"] = df_ROI_FNA_Scan
pop_column = df_ROI_FNA.pop('Scan')
df_ROI_FNA.insert(1, 'Scan', pop_column)
pop_column = df_ROI_FNA.pop('Row')
df_ROI_FNA.insert(1, 'Row', pop_column)
pop_column = df_ROI_FNA.pop('Sample')
df_ROI_FNA.insert(1, 'Sample', pop_column)

df_ROI_Di["Sample"] = 0
df_ROI_Di["Row"] = 0
df_ROI_Di["Scan"] = 0
df_ROI_Di_Sample = []
df_ROI_Di_Row = []
df_ROI_Di_Scan = []
for i in range(len(df_ROI_Di["pixel_id"])):
    df_ROI_Di_Sample.append("_".join(str(df_ROI_Di["pixel_id"][i]).split("_")[:-2]))
    df_ROI_Di_Row.append(int(str(df_ROI_Di["pixel_id"][i]).split("_")[-1]))
    df_ROI_Di_Scan.append(int(str(df_ROI_Di["pixel_id"][i]).split("_")[-2]))
df_ROI_Di["Sample"] = df_ROI_Di_Sample
df_ROI_Di["Row"] = df_ROI_Di_Row
df_ROI_Di["Scan"] = df_ROI_Di_Scan
pop_column = df_ROI_Di.pop('Scan')
df_ROI_Di.insert(1, 'Scan', pop_column)
pop_column = df_ROI_Di.pop('Row')
df_ROI_Di.insert(1, 'Row', pop_column)
pop_column = df_ROI_Di.pop('Sample')
df_ROI_Di.insert(1, 'Sample', pop_column)

In [20]:
df_ROI_train = df_ROI_train.sort_values("Sample", ascending = True).reset_index().drop(columns=["index"])
df_ROI_ext = df_ROI_ext.sort_values("Sample", ascending = True).reset_index().drop(columns=["index"])
df_ROI_FNA = df_ROI_FNA.sort_values("Sample", ascending = True).reset_index().drop(columns=["index"])
df_ROI_Di = df_ROI_Di.sort_values("Sample", ascending = True).reset_index().drop(columns=["index"])

In [21]:
df_ROI_Di

,pixel_id,Sample,Row,Scan,311.1684,269.2486,215.0328,295.2278,883.533,309.1704,738.5066,435.2965,250.1449,514.2846,type,predicted0n1,T,F,CM
0,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01,1,1,0.005233,0.142288,0.039170,0.003879,0.077996,0.211155,0.061852,0.125873,0.467335,0.036498,0,0,0,1,TN
1,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01,1,400,0.055162,0.307362,0.128310,0.049762,0.115976,0.285178,0.139311,0.244476,0.548796,0.067742,0,0,0,1,TN
2,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01,1,401,0.082919,0.122827,0.154796,-0.002981,0.155022,0.322076,0.093482,0.161324,0.551914,0.100333,0,0,0,1,TN
3,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01,1,402,0.029735,0.280930,0.082695,0.045663,0.159267,0.206077,0.159571,0.261497,0.492367,0.043914,0,0,0,1,TN
4,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01,1,403,0.107116,0.286127,0.043843,0.064159,0.130098,0.344912,0.117790,0.177373,0.568551,0.094227,0,0,0,1,TN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6052,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S1...,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12,1,203,-0.329499,-0.315904,-0.308808,-0.344216,-0.273013,-0.362289,-0.280535,-0.277840,-0.089381,-0.271914,1,1,1,0,TP
6053,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S1...,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12,1,204,-0.205739,-0.111411,-0.174013,-0.188862,-0.273013,-0.181581,-0.199019,0.053283,-0.032785,-0.119012,1,0,0,1,FN
6054,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S1...,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12,1,205,-0.074755,-0.036041,0.173826,-0.020632,-0.007824,-0.095057,-0.080780,0.036554,-0.007738,-0.002107,1,0,0,1,FN
6055,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S1...,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12,1,207,-0.326381,-0.259228,-0.308808,-0.313712,-0.273013,-0.294656,-0.280535,-0.277840,0.002593,-0.265437,1,1,1,0,TP


In [22]:
HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_ROI = df_ROI_train.loc[df_ROI_train["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3_ROI = df_ROI_train.loc[df_ROI_train["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2_ROI = df_ROI_train.loc[df_ROI_train["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4_ROI = df_ROI_train.loc[df_ROI_train["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5_ROI = df_ROI_train.loc[df_ROI_train["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4_ROI = df_ROI_train.loc[df_ROI_train["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1_ROI = df_ROI_ext.loc[df_ROI_ext["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2_ROI = df_ROI_ext.loc[df_ROI_ext["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3_ROI = df_ROI_ext.loc[df_ROI_ext["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4_ROI = df_ROI_ext.loc[df_ROI_ext["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_ROI = df_ROI_FNA.loc[df_ROI_FNA["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3_ROI = df_ROI_FNA.loc[df_ROI_FNA["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1_ROI = df_ROI_FNA.loc[df_ROI_FNA["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])

HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1_ROI = df_ROI_train.loc[df_ROI_train["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1_ROI = df_ROI_train.loc[df_ROI_train["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4_ROI = df_ROI_train.loc[df_ROI_train["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3_ROI = df_ROI_train.loc[df_ROI_train["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6_ROI = df_ROI_train.loc[df_ROI_train["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4_ROI = df_ROI_train.loc[df_ROI_train["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1_ROI = df_ROI_ext.loc[df_ROI_ext["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2_ROI = df_ROI_ext.loc[df_ROI_ext["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3_ROI = df_ROI_ext.loc[df_ROI_ext["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4_ROI = df_ROI_ext.loc[df_ROI_ext["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_ROI = df_ROI_FNA.loc[df_ROI_FNA["Sample"] == "HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5_ROI = df_ROI_FNA.loc[df_ROI_FNA["Sample"] == "HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2_ROI = df_ROI_FNA.loc[df_ROI_FNA["Sample"] == "HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])

HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01_ROI = df_ROI_Di.loc[df_ROI_Di["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S04_ROI = df_ROI_Di.loc[df_ROI_Di["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S04"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S05_ROI = df_ROI_Di.loc[df_ROI_Di["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S05"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S10_ROI = df_ROI_Di.loc[df_ROI_Di["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S10"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S12_ROI = df_ROI_Di.loc[df_ROI_Di["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S12"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01_ROI = df_ROI_Di.loc[df_ROI_Di["Sample"] == "HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S04_ROI = df_ROI_Di.loc[df_ROI_Di["Sample"] == "HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S04"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S10_ROI = df_ROI_Di.loc[df_ROI_Di["Sample"] == "HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S10"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S11_ROI = df_ROI_Di.loc[df_ROI_Di["Sample"] == "HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S11"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12_ROI = df_ROI_Di.loc[df_ROI_Di["Sample"] == "HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])


In [23]:
sample_ROI_list = [HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4_ROI, 
                   HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_ROI, 
                   HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5_ROI, 
                   HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S04_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S05_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S10_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S12_ROI, 
                   HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01_ROI, 
                   HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S04_ROI, 
                   HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S10_ROI, 
                   HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S11_ROI, 
                   HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12_ROI]

In [24]:
name_list = ["HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4_ROI", 
             "HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_ROI", 
             "HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5_ROI", 
             "HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S04_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S05_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S10_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S12_ROI", 
             "HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01_ROI", 
             "HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S04_ROI", 
             "HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S10_ROI", 
             "HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S11_ROI", 
             "HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12_ROI"]

In [25]:
sample_ROI_list_train = [HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_ROI, 
                         HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3_ROI, 
                         HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2_ROI, 
                         HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4_ROI, 
                         HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5_ROI, 
                         HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4_ROI, 
                         HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1_ROI, 
                         HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1_ROI, 
                         HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4_ROI, 
                         HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3_ROI, 
                         HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6_ROI, 
                         HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4_ROI]

sample_ROI_list_ext = [HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1_ROI, 
                       HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2_ROI, 
                       HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3_ROI, 
                       HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4_ROI, 
                       HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1_ROI, 
                       HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2_ROI, 
                       HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3_ROI, 
                       HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4_ROI]

sample_ROI_list_ingested = [HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4_ROI]

sample_ROI_list_FNA = [HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_ROI, 
                       HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3_ROI, 
                       HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1_ROI, 
                       HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_ROI, 
                       HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5_ROI, 
                       HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2_ROI]

sample_ROI_list_DirectIn = [HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S04_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S05_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S10_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S12_ROI, 
                            HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01_ROI, 
                            HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S04_ROI, 
                            HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S10_ROI, 
                            HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S11_ROI, 
                            HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12_ROI]

In [26]:
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_ROI

,pixel_id,Sample,Row,Scan,311.1684,269.2486,215.0328,295.2278,883.533,309.1704,738.5066,435.2965,250.1449,514.2846,type,predicted0n1,T,F,CM
0,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,1,4,0.113060,-0.163063,0.249717,0.023973,-0.010818,0.208528,-0.400766,-0.215854,0.047157,0.153808,0,1,1,0,FP
1,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,1,5,-0.005655,0.069112,0.241310,0.088081,-0.027146,0.194772,0.006021,0.034379,0.128858,0.113312,0,0,0,1,TN
2,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,1,6,-0.005655,0.101541,0.247080,0.045869,-0.053504,0.181977,-0.400766,0.068699,0.156587,0.154891,0,0,0,1,TN
3,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,1,194,-0.005655,0.663453,0.445648,0.574315,0.008805,0.638267,0.197404,0.451272,0.689391,0.388039,0,0,0,1,TN
4,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,1,195,-0.005655,0.603402,0.522501,0.639386,-0.102022,0.648668,0.248245,0.353670,0.592447,0.492495,0,0,0,1,TN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14820,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,117,144,-0.005655,0.114157,0.121770,0.073214,0.035831,0.088039,-0.017831,0.252176,-0.052651,0.053597,0,0,0,1,TN
14821,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,117,145,-0.005655,0.110845,0.150544,0.138104,-0.053143,-0.033029,-0.074229,0.193749,0.005182,0.101802,0,0,0,1,TN
14822,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,117,146,-0.005655,0.097728,0.175508,0.149865,-0.141480,-0.033029,0.047959,0.200532,-0.021802,0.056586,0,0,0,1,TN
14823,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,118,177,-0.005655,0.482772,-0.225954,0.283049,-0.178133,0.225924,-0.124501,0.075612,0.344417,-0.039099,0,0,0,1,TN


In [27]:
HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_ROI

,pixel_id,Sample,Row,Scan,311.1684,269.2486,215.0328,295.2278,883.533,309.1704,738.5066,435.2965,250.1449,514.2846,type,predicted0n1,T,F,CM
0,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_5_1,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3,1,5,0.751601,0.209629,-0.225954,0.051292,0.006771,0.848857,-0.098114,-0.215854,0.325501,-0.039099,1,0,0,1,FN
1,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_6_1,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3,1,6,0.615763,0.278727,-0.225954,-0.152267,0.126440,0.693306,-0.022942,-0.215854,0.379190,-0.039099,1,0,0,1,FN
2,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_7_1,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3,1,7,0.573044,0.293220,-0.225954,0.048487,0.136786,0.581716,0.077861,-0.215854,0.430778,-0.039099,1,0,0,1,FN
3,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_8_1,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3,1,8,0.597390,0.364789,-0.225954,0.053772,0.000260,0.605680,0.147088,-0.215854,0.419034,0.171491,1,0,0,1,FN
4,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_9_1,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3,1,9,0.649070,0.313500,-0.225954,0.097820,-0.008422,0.686924,-0.004725,-0.215854,0.455942,-0.039099,1,0,0,1,FN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14249,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_1...,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3,125,189,0.201315,0.374268,-0.225954,0.214496,-0.025201,-0.033029,-0.400766,-0.215854,0.461811,-0.039099,1,0,0,1,FN
14250,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_1...,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3,125,190,0.206287,0.344487,-0.225954,0.175976,0.057545,-0.033029,-0.400766,-0.215854,0.481184,-0.039099,1,0,0,1,FN
14251,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_1...,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3,125,191,0.201992,0.359319,-0.225954,0.194592,-0.005625,-0.033029,-0.400766,-0.215854,0.528552,-0.039099,1,0,0,1,FN
14252,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_1...,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3,126,185,0.199953,0.388687,-0.225954,0.199553,0.104115,-0.033029,-0.400766,-0.215854,0.522575,-0.039099,1,0,0,1,FN


# 2. Prediction (made)

## 2.1. FNA

In [28]:
( list(HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_ROI["predicted0n1"]).count(1)) ) *100, "%"

(46.61720067453626, '%')

In [29]:
( list(HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3_ROI["predicted0n1"]).count(1)) ) *100, "%"

(50.32842425499551, '%')

In [30]:
( list(HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1_ROI["predicted0n1"]).count(1)) ) *100, "%"

(23.3739837398374, '%')

In [31]:
( list(HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_ROI["predicted0n1"]).count(1)) ) *100, "%"

(78.73579346148449, '%')

In [32]:
( list(HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5_ROI["predicted0n1"]).count(1)) ) *100, "%"

(74.88720760741306, '%')

In [33]:
( list(HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2_ROI["predicted0n1"]).count(1)) ) *100, "%"

(76.65806451612903, '%')

## 2.2. Direct Infusion

In [34]:
( list(HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01_ROI["predicted0n1"]).count(1)) ) *100, "%"

(3.814262023217247, '%')

In [35]:
( list(HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S04_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S04_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S04_ROI["predicted0n1"]).count(1)) ) *100, "%"

(62.541254125412536, '%')

In [36]:
( list(HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S05_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S05_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S05_ROI["predicted0n1"]).count(1)) ) *100, "%"

(24.422442244224424, '%')

In [37]:
( list(HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S10_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S10_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S10_ROI["predicted0n1"]).count(1)) ) *100, "%"

(26.897689768976896, '%')

In [38]:
( list(HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S12_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S12_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S12_ROI["predicted0n1"]).count(1)) ) *100, "%"

(86.63366336633663, '%')

In [39]:
( list(HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01_ROI["predicted0n1"]).count(1)) ) *100, "%"

(34.15841584158416, '%')

In [40]:
( list(HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S04_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S04_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S04_ROI["predicted0n1"]).count(1)) ) *100, "%"

(73.10231023102311, '%')

In [41]:
( list(HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S10_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S10_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S10_ROI["predicted0n1"]).count(1)) ) *100, "%"

(61.221122112211226, '%')

In [42]:
( list(HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S11_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S11_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S11_ROI["predicted0n1"]).count(1)) ) *100, "%"

(62.211221122112214, '%')

In [43]:
( list(HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12_ROI["predicted0n1"]).count(1)) ) *100, "%"

(88.77887788778878, '%')

# 3. EDA

In [44]:
from matplotlib.pyplot import figure
plt.rcParams['figure.figsize'] = [10, 10]
plt.grid(visible=None)
plt.close()
figure(figsize=(10, 10))

<Figure size 720x720 with 0 Axes>

In [45]:
import matplotlib.pyplot as plt
import matplotlib.colors
import matplotlib.cm
import numpy as np
from matplotlib.colors import ListedColormap
cmap = cm.binary

def rgb_white2alpha(rgb, ensure_increasing=False):

    # The most transparent alpha we can use is given by the min of RGB
    # Convert it from saturation to opacity
    alpha = 1. - np.min(rgb, axis=1)
    if ensure_increasing:
        # Let's also ensure the alpha value is monotonically increasing
        a_max = alpha[0]
        for i, a in enumerate(alpha):
            alpha[i] = a_max = np.maximum(a, a_max)
    alpha = np.expand_dims(alpha, -1)
    # Rescale colors to discount the white that will show through from transparency
    rgb = (rgb + alpha - 1) / alpha
    # Concatenate our alpha channel
    return np.concatenate((rgb, alpha), axis=1)
    

def cmap_white2alpha(name, ensure_increasing=False, register=True):

    # Fetch the cmap callable
    cmap = plt.get_cmap(name)
    # Get the colors out from the colormap LUT
    rgb = cmap(np.arange(cmap.N))[:, :3]  # N-by-3
    # Convert white to alpha
    rgba = rgb_white2alpha(rgb, ensure_increasing=ensure_increasing)
    # Create a new Colormap object
    cmap_alpha = matplotlib.colors.ListedColormap(rgba, name=name + "_alpha")
    if register:
        matplotlib.cm.register_cmap(name=name + "_alpha", cmap=cmap_alpha)
    return cmap_alpha

In [60]:
# Get our new version of the colormap with white2alpha
cmap_w2a = cmap_white2alpha("Reds")

In [61]:
def genAROItif(sample_ROI, sample_, name, mz, vmax_=1):

    df_heatmap = sample_ROI[["Row", "Scan", mz]]
    df_heatmap2 = pd.DataFrame(np.array([[0, 0, 0]]), columns=['Row', 'Scan', mz]).reset_index().drop(columns = ["index"])
    df_HM = df_heatmap2.loc[df_heatmap["Row"] == 0].loc[df_heatmap2["Scan"] == 0]
    Heat_Row = sample_["Row"]
    Heat_Scan = sample_["Scan"]


    #hi = 0
    for y in range(len(set(Heat_Row))):
        for x in range(1, len(set(Heat_Scan))+1):

            df_in = df_heatmap.loc[df_heatmap["Row"] == (y+1)].loc[df_heatmap["Scan"] == x]
            if len(df_in) == 1:
                df_HM = pd.concat([df_HM, df_in], join="inner", ignore_index=True)
                df_HM = df_HM.sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns = ["index"])
                #hi += 1


            elif len(df_in) == 0:
                df_heatmap2["Row"] = y+1
                df_heatmap2["Scan"] = x
                df_HM = pd.concat([df_HM, df_heatmap2], join="inner", ignore_index=True)
                df_HM = df_HM.sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns = ["index"])
                #hi += 1


    #print(name, mz, hi, len(df_HM))
    if "index" in df_HM.columns:
        df_HM = df_HM.drop(columns = ["index", "Row", "Scan"])
    else:
        df_HM = df_HM.drop(columns = ["Row", "Scan"])

    df_HM = df_HM.T
    New_y = []
    for y in range(1, len(set(Heat_Row))+1):
        New_y.append(str(y))
    New_x = []
    for x in range(1, len(set(Heat_Scan))+1):
        New_x.append(str(x))

    arr = df_HM.values.copy()
    arr.resize(len(set(Heat_Row)), len(set(Heat_Scan)))

    df_HM = pd.DataFrame(arr)
    df_HM.index = New_y
    df_HM.columns = New_x

    for i in list(df_HM.columns):
        df_HM[i] = df_HM[i].astype(float)

    plt.subplots(dpi = 300)
    plt.imshow(df_HM, cmap=cmap_w2a, vmin=0, vmax=1, zorder=1) #cmap_w2a
    plt.xticks([])
    plt.yticks([])
    plt.box(False)
    plt.autoscale(enable=True, tight=True)

    plt.rcParams['figure.figsize'] = [2, 2]

    plt.savefig(f"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/XGB_ALL/afterModelSelection/dbMSIn_10ppm/{name}_{mz}_1.tif", bbox_inches = 'tight', pad_inches = 0)

In [49]:
df_raMSI_ML = pd.read_csv(r'H:/3_output_raMSIn/3_3_Output_raMSI_HKU_Ingested4FNA/0_dbMSI_10ppm/dbMSI10ppm_noLT.csv').set_index('pixel_id')
df_raMSI_ML.shape

(371480, 2603)

In [50]:
raMSI_ML = df_raMSI_ML.reset_index()

In [51]:
raMSI_ML["Sample"] = 0
raMSI_ML["Row"] = 0
raMSI_ML["Scan"] = 0
Sample = []
Row = []
Scan = []
for i in range(len(raMSI_ML["pixel_id"])):
    Sample.append("_".join(str(raMSI_ML["pixel_id"][i]).split("_")[:-2]))
    Row.append(int(str(raMSI_ML["pixel_id"][i]).split("_")[-1]))
    Scan.append(int(str(raMSI_ML["pixel_id"][i]).split("_")[-2]))
raMSI_ML["Sample"] = Sample
raMSI_ML["Row"] = Row
raMSI_ML["Scan"] = Scan

In [52]:
pop_column = raMSI_ML.pop('Scan')
raMSI_ML.insert(1, 'Scan', pop_column)
pop_column = raMSI_ML.pop('Row')
raMSI_ML.insert(1, 'Row', pop_column)
pop_column = raMSI_ML.pop('Sample')
raMSI_ML.insert(1, 'Sample', pop_column)

In [53]:
raMSI_ML = raMSI_ML.sort_values("Sample", ascending = True).reset_index().drop(columns=["index"])

In [54]:
HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])

HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])

In [55]:
sample_list = [HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2, 
               HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3, 
               HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2, 
               HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4, 
               HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5, 
               HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4, 
               HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1, 
               HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2, 
               HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3, 
               HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4, 
               HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3, 
               HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3, 
               HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1, 
               HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1, 
               HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1, 
               HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4, 
               HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3, 
               HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6, 
               HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4, 
               HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1, 
               HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2, 
               HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3, 
               HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4, 
               HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3, 
               HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5, 
               HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2]

In [62]:
import warnings
warnings.filterwarnings('ignore')

from datetime import datetime

TOP ="T"
    

counter = 0

for im in range(len(sample_ROI_list[:-10])):
    start = datetime.now()

    name = name_list[im]
    sample_ = sample_list[im]
    preview = sample_ROI_list[im][[TOP]]
    for i in preview.columns:
        preview[i] = preview[i].astype(float)

    vmax_list = list(preview.describe().iloc[-1])
    vmax_ = 1
    #print(TOP, name, vmax_)

    genAROItif(sample_ROI=sample_ROI_list[im], sample_=sample_, name=name, mz=TOP, vmax_=vmax_)
    #print(df.shape)
    counter += 1
    end = datetime.now()
    Duration = end - start
    print(Duration)


0:00:49.870126
0:00:55.591367
0:01:04.284450
0:00:34.643870
0:00:40.994193
0:00:56.013745
0:00:03.439451
0:00:02.412118
0:00:03.829296
0:00:03.655223
0:02:02.647831
0:01:58.013080
0:02:07.891656
0:00:57.442386
0:00:48.299735
0:00:40.872725
0:00:40.513976
0:00:39.516764
0:00:39.529060
0:00:02.444534
0:00:05.824844
0:00:03.203214
0:00:02.774544
0:01:54.221683
0:01:55.996497
0:02:11.321320
